a/b testing for new models using metric: maximize precision score and output raw probabilities, get those from pred proba

In [1]:
import pandas as pd
import numpy as np
from joblib import dump, load

In [2]:
X_train_sc = pd.read_csv('X_train_sc.csv')
X_val_sc = pd.read_csv('X_val_sc.csv')
y_train_enc = np.ravel(pd.read_csv('y_train_enc.csv'))
y_val_enc = np.ravel(pd.read_csv('y_val_enc.csv'))


In [3]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_train_sc, y_train_enc = sm.fit_resample(X_train_sc,y_train_enc)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report


In [5]:
def modeler(est,X_train,y_train,X_val,y_val):
    
    est.fit(X_train,y_train)
    
    
    train_acc_score = np.mean(cross_val_score(est,X_train,y_train,scoring='precision',cv=7,n_jobs=4))
    val_acc_score = np.mean(cross_val_score(est,X_val,y_val,scoring='precision',cv=7,n_jobs=4))
    diff = abs(train_acc_score - val_acc_score)
    
    y_val_pred = est.predict(X_val)
    
    
    print(f'train precision: {round(train_acc_score,5)}')
    print('-'*40)
    print(f'val precision: {round(val_acc_score,5)}')
    print('-'*40)
    print(f'difference: {round(diff,7)}')
    print('-'*40)
    print(f'val report: ')
    print(classification_report(y_val,y_val_pred))
    print('-'*40)
    return est

Get probabilities for most likely to churn

In [9]:
lr = LogisticRegression()

est = modeler(lr,X_train_sc,y_train_enc,X_val_sc,y_val_enc)

train precision: 0.77144
----------------------------------------
val precision: 0.65911
----------------------------------------
difference: 0.1123231
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.90      0.73      0.81       971
           1       0.51      0.77      0.61       350

    accuracy                           0.74      1321
   macro avg       0.70      0.75      0.71      1321
weighted avg       0.79      0.74      0.75      1321

----------------------------------------


In [13]:
pred_probs = pd.DataFrame(est.predict_proba(X_val_sc))

In [32]:
pred_probs.groupby(pred_probs[1]).sum().nsmallest(20,columns=0)

,0
1,
0.980881,0.019119
0.979518,0.020482
0.976344,0.023656
0.975200,0.024800
0.972325,0.027675
0.972069,0.027931
0.967254,0.032746
0.966392,0.033608
0.963226,0.036774
